# 本文为中国农业大学 图算法与机器学习研究小组研讨会 第三期课堂内容
### **本作品采用 CC-BY-SA 4.0协议 进行许可。**


# Presented By Endurernura (Ryan Woo)
### 本文作者：Ryan (吴秉政)

### 接下来，我将向你展示如何通过pytorch（CPU），通过全连接神经网络实现28x28图像的手写数字识别任务。
我们首先import需要用到的第一个库。这里的**torch.nn**包含了一个神经网络的**核心部分**需要的所有内容。<br>
**torch.optim**包括有你常用的优化器，如SGD，Adam等。*至于你的损失函数，其实它一般在 **torch.nn** 里面。*<br>
**torch.ultils.data**中包括所有与**数据加载**相关的组件。
torch为你的基础模型提供了许多不同方向、不同类型的训练数据。在我们的这个任务中，**torchvision**库里有我们需要的MNIST数据集。关于数据集的更多内容在[这里](https://zhuanlan.zhihu.com/p/155748813)。

注意：**本文中的代码由于需要满足“理解性优先”的原则，*不能直接运行*。你需要删除一些内容，或者去**[这里](https://github.com/Endurernura/CAU-GLC-Programme)。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import dataloader, dataset
from torchvision import datasets, transforms

接下来我们开始设计这个全连接神经网络的核心部分。<br>
不妨假设一个像素上的灰度作为第一层（输入层）的神经元数量。这里我们已经算出来了，28 x 28 = 784，也就是说第一层我们需要784个神经元。<br>
pytorch的语法设计非常人性化——我们稍后再来讨论这一点。<br>
让我们继续设计。我觉得这个任务并不是特别难，而且我们也并不需要一个性能多么强大的模型，第二层就不妨把它设为196个神经元(196 * 4 = 784)。<br>
第三层，我们希望有十个输出，这样根据每一个神经元的值就可以判定这个图片中的数到底是0-9中的哪一个了。<br>
我们总结一下，刚才设计的神经网络包括一个784个神经元的输入层，一个10神经元的输出层，在这中间我们设计了一个196神经元的中间层，或者你也可以叫它hidden layer。于是，我们现在开始通过torch来实现我们的这个神经网络的主体部分。我们首先需要一个类，叫它fc即可。

In [ ]:
class fc(nn.module):
    def __init__(self):
        super(fc, self).__init__()
        self.layer1 = nn.linear(784, 196)
        self.hidden_layer = nn.linear(196, 10)
        self.relu = nn.relu()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        return x

### 你可能懵了：这都是什么东西？别急，让我们一点一点展开。<br>
我们新定义一个类，称作fc (fully connection network)。这个类必须继承nn.module，这样才能用torch.nn里面给你的东西。

In [ ]:
class fc(nn.module):

这里仍然是关于“类”的内容。这里想要展开解释并不容易，你可以认为第一行是任何一个类都要有的初始化过程，而第二行 (super...) 是为了与父类（nn.module）建立联系。

In [ ]:
def __init__(self):
    super().__init__()

接下来，我们按照之前的设计完成神经网络。还记得每一层有多少个神经元吗？别忘了激活函数。没有激活函数，神经网络几乎无法工作。

In [ ]:
self.layer1 = nn.linear(784, 196)
self.hidden_layer = nn.linear(196, 10)
self.relu = nn.relu()

我们接下来完成模型的前向传播部分。还记得之前讲过的，什么是前向传播吗？

In [ ]:
def foward(self, x):
    x = x.view(-1, 784)
    x = self.layer1(x)
    x = self.relu(x)
    x = self.hidden_layer(x)
    return x

### OK! 好的，我们结束了核心部分。接下来该干什么？
当然是给模型喂数据! 老样子，我先在这里展示数据输入部分的全部代码，然后我们一点一点拆解。

In [ ]:
from torch.utils.data import dataloader, dataset
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(),
transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    root='./data', 
    train=True,
    download=True,
    transform=transform
)
test_dataset = datasets.MNIST(
    root='./data', 
    train=False,
    transform=transform
)


train_loader = dataloader(train_dataset, batch_size=64, shuffle=True)
test_loader = dataloader(test_dataset, batch_size=1000)

一开始的import部分自然不用说。torch为初学者提供了很多可以直接拿来用的数据，比如**torchvision**和**torchaudio**，以及其他（我没用过所以不知道，这里可以自行百度）。我们这里用到的MNIST数据集就通过torchvision.dataset.MNIST获得，它在torchvision.data里以函数形式存在，在使用它的时候直接赋值给变量就可以。注意这里的参数*root, train*和*transform*，它们的用法就不用多说，一目了然。


In [ ]:
train_dataset = datasets.MNIST(
    root='./data', 
    train=True,
    download=True,
    transform=transform
)
test_dataset = datasets.MNIST(
    root='./data', 
    train=False,
    transform=transform
)

接下来我们说数据预处理部分。MNIST数据集中的原始数据是一堆图像，我们需要把图像处理成程序能够处理的张量（tensor）形式，也就是用到了**transforms.ToTensor()**。<br>
随后，**transforms.Normalize**用来将数据归一化，也就是通过归一化将所有的灰度数据压缩到[-1, 1]的范围内，表现为
**((0.5,), (0.5,))**。

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

*别问我为什么归一化能防止梯度爆炸。我也不知道。*

Anyway, 后面的操作就很简单了。我们最终获得了训练数据加载器**train_loader**和验证数据加载器**test_loader**。

回忆一下，对于一个机器学习模型，除了它的核心部分和数据输入部分，我们还需要什么?<br>
对了。那就是损失函数和优化器。我们这里是一个多分类任务，因此采取最经典的配置方法，交叉熵损失函数（CrossEntryLoss）和Adam优化器。关于优化器和损失函数，更多的内容请参见西瓜书。

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fc.parameters(), lr=0.001)

for epoch in range(5):

    for images, labels in train_loader:
        outputs = fc(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()   
        optimizer.step()
        
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

顾名思义，citerion和optimizer是损失函数和优化器的加载语句。它们的用法也非常一目了然。

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fc.parameters(), lr=0.001)

对于训练过程，最直观的想法就是用循环来完成。计算机语言基础的部分我们不做讲解。

对于训练数据集中的每个图片和标签而言，我们把图片数据（实际上是一个torch张量）喂给模型，初始化优化器 (optimizer.zero_grad 零梯度。为什么是零梯度呢？自己想一下)。

In [ ]:
for epoch in range(5):

    for images, labels in train_loader:
        outputs = fc(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()   
        optimizer.step()

好了。我们现在把所有的东西放在一起。<br>
如果你想，下面的代码可以直接运行。

In [ ]:
import torch
import torch.nn as nn

class fc(nn.Module):
    def __init__(self):
        super(fc, self).__init__()
        self.layer1 = nn.Linear(784, 196)
        self.hidden_layer = nn.Linear(196, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        return x
fc = fc()

from torch.utils.data import DataLoader, dataset
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(
    root='./data', 
    train=True,
    download=True,
    transform=transform
)
test_dataset = datasets.MNIST(
    root='./data', 
    train=False,
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fc.parameters(), lr=0.001)

for epoch in range(5):

    for images, labels in train_loader:
        outputs = fc(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()   
        optimizer.step()
        
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

一件非常搞笑的事情是，你在阅读的时候可能会发现一个问题。为什么前面的代码运行不了呢？<br>
其实我整理的时候，在自己试着跑一边这个完整模型的时候也发现了这样的问题。但是，python（我认为）的一个很大的优点就是，相比C来说，它的规范更容易被认为修正得更加“舒服”。<br>
比如说，在运行问题报错时，编译器本身就会告诉你错在了哪里（其实应该叫它解释器，但是我习惯了，而且不想改。 ：-D）。刚才前面模块化的部分就出了很多问题。你能找到它们吗？第一个找全问题并e-mail我的，下周的讨论会上将会获得我送出的小礼物一件。<br>

(BTW：不要太期待，我很穷。)<br>
总之，Thanks for reading!<br>
我们下周再见。